# Discrete State Seeker
Consider a robot with state variable $x = (x_0,x_1,...)$ where $x_t \in \mathcal{X}$ for every step $t \in \mathbb{N}$. At every step $t$, the robot can choose an action $a_t \in \mathcal{A}$ according to policy $a_t^\pi=\pi(x_t)$ that stochastically sets the robot's state to $x_{t+1}=Δ(x_t,a)$ and earns some reward $r(x_t,x_{t+1},a)$. The robot's goal is to maximize the total reward over an infinite time horizon with discount factor $\rho$: $\pi^\star = \argmax_{\pi}{\sum_{t=0}^{\infty} {\rho^t r(x_t, x_{t+1}, a_t^\pi)}}$.

We will use the off-policy TD algorithm (Q-learning) to estimate $Q(x,a)$, the expected reward remaining when taking action $a$ at state $x$; the $\pi^\star$ will then simply be to choose the action $a$ that maximizes $Q(x,a)$ at any state $x$. We will achieve this through:
1. Let $\alpha = (\alpha_0, \alpha_1, ...)$ be the learning rate with $\alpha_n \in (0,1)$, $\sum_n \alpha_n = \infty$, $\sum_n \alpha_n^2 < \infty$
2. Randomly generate an initial $Q_0$ matrix and an initial $x_0$. Choose some randomized policy $\pi$.
3. For $n=0,1,2,...$
    - Suppose $Q_n$ and $x_n$ are known. Choose action $a_n$ according to $\pi$ and $x_{n+1} = Δ(x_n,a_n)$.
    - Calculate updated values for $Q_{n+1}(x_n,a_n)$ based on the following equation:
        $$Q_{n+1}(x_n,a_n) = Q_n(x_n,a_n) + \alpha_n (r(x_n, x_{n+1}, a_n) + \rho \sup_{b \in \mathcal{A}} Q_n(x_{n+1},b) - Q_n(x_n,a_n))$$
4. Repeat the recursion until $Q_{n+1} \approx Q_n$.

In [68]:
import numpy as np
from matplotlib import pyplot as plt

In [69]:
class State_Seeker:
    def __init__(self, state_space, action_space, discount, next_state, reward):
        self.state_space = state_space # \mathcal{X}  = list of possible states
        self.action_space = action_space # \mathcal{A} = list of possible actions at any state
        self.discount = discount # \rho = the discount factor
        self.next_state = next_state # Δ(x_n,a) = the callable that stochastically determines x_{n+1}
        self.reward = reward # r(x_n, x_{n+1}, a) = the reward for going from state x_n to x_{n+1} through action a

        # other instance variables
        self.Q = np.zeros(shape=(len(state_space), len(action_space)))

    # resets the Q matrix to a randomized matrix
    # Alternatively, can provide a Q to set the matrix to
    # Or can provide a range to randomly select element values from
    def reset_Q(self, Q=None, range=[0,1]):
        if Q is not None:
            self.Q = np.random.rand(len(self.state_space), len(self.action_space)) * (range[1] - range[0]) + range[0]

    # runs the Q-learning algorithm
    # we choose x_0 randomly and use a uniformly random policy for training
    # if plot=True, plots the sum of the squares of the Q matrix elements over the steps
    def train_Q(self, learning_rate, min_iterations, tolerance, test_increment=None, test_horizon=None, test_trials=None):

        if test_increment != None:
            performance = []

        x = np.random.choice(self.state_space)
        step = 0
        while step < min_iterations or np.abs(Q_change) > tolerance:
            if test_increment != None and step % test_increment == 0:
                performance.append(self.test_performance(test_horizon=test_horizon, trials=test_trials)[0])

            # determine a and x_{n+1}
            a_index = np.random.randint(0, len(self.action_space))
            a = self.action_space[a_index]
            x_next = self.next_state(x, a)
            x_index = self.state_space.index(x)
            x_next_index = self.state_space.index(x_next)

            # find Q_{n+1}
            Q_change = learning_rate(step) * (
                self.reward(x, x_next, a) +
                self.discount * np.max(self.Q[x_next_index,]) -
                self.Q[x_index,a_index]
            )

            # set Q = Q_{n+1} and x = x_next
            self.Q[x_index, a_index] += Q_change
            x = x_next
            step += 1
            
        if test_increment != None:
            plt.plot(performance)
            plt.xlabel(f"Q-Learning Step ({test_increment}s)")
            plt.ylabel("Estimated Reward")
        return self.Q, step

    # returns the value function at state x and the best action to take
    def value_action(self, x):
        best_index = np.argmax(self.Q[self.state_space.index(x),])
        return self.Q[self.state_space.index(x),best_index], self.action_space[best_index]
    
    # runs a given number of trials of the policy
    # return the mean and stdev of the total discounted rewards
    def test_performance(self, test_horizon, initial_state=None, trials=1):
        results = np.zeros(trials)
        for trial in range(trials):
            x = initial_state if initial_state != None else np.random.choice(self.state_space)
            total_reward = 0
            for step in range(test_horizon):
                _,a = self.value_action(x)
                x_next = self.next_state(x,a)
                total_reward += self.discount**step * self.reward(x, x_next, a)
            results[trial] = total_reward
        return np.mean(results), np.std(results)

In [74]:
recycler_states = ["low", "high"]
recycler_actions = ["search", "wait", "charge"]

def recycler_next_state(x,a):
    p_h = 0.8 # probability of remaining high battery upon searching
    p_l = 0.7 # probability of remaining low battery upon searching
    # (x, x_next, action) -> probability
    prob_matrix = np.array([
            [
                [p_l,    1.0,    0.0], # low to low
                [1-p_l,  0.0,    1.0], # low to high
            ], [
                [1-p_h,  0.0,    0.0], # high to low
                [p_h,    1.0,    1.0], # high to high
            ],
        ])
    return np.random.choice(recycler_states, p=prob_matrix[recycler_states.index(x),:,recycler_actions.index(a)])

def recycler_reward(x,x_next,a):
    r_s = 1 # reward for searching
    r_w = 0.1 # reward for waiting
    r_p = -10 # reward for running out of battery (low -> high)
    # (x, x_next, action) -> reward
    reward_matrix = np.array([
            [
                [r_s,   r_w,     np.nan], # low to low
                [r_p,   np.nan,  0], # low to high
            ], [
                [r_s,   np.nan,  np.nan], # high to low
                [r_s,   r_w,     0], # high to high
            ],
        ])
    return reward_matrix[recycler_states.index(x),recycler_states.index(x_next),recycler_actions.index(a)]

recycler = State_Seeker(
    state_space = recycler_states, # low, high
    action_space = recycler_actions, # search, wait, charge
    discount = 0.9,
    next_state = recycler_next_state,
    reward = recycler_reward,
)

recycler.reset_Q()
recycler.train_Q(
    learning_rate = lambda n : 1/(1+0.1*n),
    min_iterations = 10**5,
    tolerance = 10**(-2),
    # test_increment=10,
    # test_horizon=100,
    # test_trials=100,
)

print(f"Q matrix:\n{recycler.Q}\n")
for state in recycler_states:
    value, best_action = recycler.value_action(state)
    print("Max value of state `%s` is %f, achieved by taking action `%s`." % (state, value, best_action))

Q matrix:
[[1.58548494 3.52843201 4.90503939]
 [6.36658426 5.73938151 5.64021668]]

Max value of state `low` is 4.905039, achieved by taking action `charge`.
Max value of state `high` is 6.366584, achieved by taking action `search`.
